In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys 
sys.path.append("../")

In [5]:
from RNN_utils import get_deep_rnn, chars_to_one_hot, sample
import numpy as np

Using TensorFlow backend.


In [6]:
model=get_deep_rnn((100, 69), dense_units=69, stateful=True)

input shape =  (100, 69)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (1, 100, 200)             216000    
_________________________________________________________________
lstm_2 (LSTM)                (1, 200)                  320800    
_________________________________________________________________
dense_1 (Dense)              (1, 69)                   13869     
Total params: 550,669
Trainable params: 550,669
Non-trainable params: 0
_________________________________________________________________


## Cuanto vale T y cuanto vale D?

In [7]:
layer1 = model.get_layer('lstm_1')
layer1.get_config()['batch_input_shape']

(1, 100, 69)

In [8]:
chars = np.load('training_chars.npy')
chars_to_indices = dict((c, i) for i, c in enumerate(chars))
indices_to_chars = dict((i, c) for i, c in enumerate(chars))

In [9]:
window = 100

In [87]:
# model.load_weights('bodes_sangre_LSTM_deep_dropout_04.hdf5')
model.load_weights('bodes_sangre_LSTM_deep_dropout_05.hdf5')

In [88]:
def predict_phrase(model, initial_seed, window = 100, verbose=True, sample_type = 'greedy'):
    # El numeral no esta en el diccionario por lo que lo pasa a todos ceros en el one-hot
    initial_seed=initial_seed[-window:]
    initial_text = '#'*(window-len(initial_seed))+initial_seed # Padding
    X_test = chars_to_one_hot(initial_seed, chars, chars_to_indices, 100)
    probs = model.predict(X_test)
    probs_sorted_indexes = np.argsort(probs)[0][::-1]
    if verbose:
        print([(indices_to_chars[i], probs[0][i]) for i in probs_sorted_indexes[:5]])
    return probs[0]

In [89]:
def sample_probs(probs, sample_type='greedy', temp=1):
    if (sample_type=='greedy'):
        return np.argmax(probs)
    if (sample_type=='simple'):
        return sample(probs, temp, verbose=False, return_dist=True)

## Distribución de probabilidades para:
#### "Estaba el novi"

In [90]:
probs = predict_phrase(model, "Estaba el novi")

[('o', 0.96835536), ('a', 0.025064146), ('e', 0.0026970813), ('l', 0.0011617033), ('t', 0.0008459071)]


In [91]:
indices_to_chars[sample_probs(probs, sample_type='greedy')]

'o'

In [92]:
temp = 1
index, dist = sample(probs, temp, verbose=False, return_dist=True)
sorted_idxs = np.argsort(dist)[::-1]
print(dist[sorted_idxs][:3])
samples = []
for i in range(20):
    index, dist = sample_probs(probs, sample_type='simple', temp=temp)
    samples.append(indices_to_chars[index])
print(samples)

[0.9683554  0.02506415 0.00269708]
['o', 'o', 'o', 'o', 'o', 'o', 'o', 'a', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']


## Distribución de probabilidades para:
#### "Estaba la novi"

In [93]:
probs = predict_phrase(model, "Estaba la novi")
indices_to_chars[sample_probs(probs, sample_type='greedy')]

[('a', 0.9924321), ('o', 0.0061247447), ('e', 0.00093063264), ('l', 0.0001637402), ('i', 6.682282e-05)]


'a'

### Mismo ejemplo pero cambiar temperatura de ruido
Notar que muestramos (Cambiar temperatura)

In [94]:
temp = 1
index, dist = sample(probs, temp, verbose=False, return_dist=True)
sorted_idxs = np.argsort(dist)[::-1]
print(dist[sorted_idxs][:3])
samples = []
for i in range(20):
    index, dist = sample_probs(probs, sample_type='simple', temp=temp)
    samples.append(indices_to_chars[index])
print(samples)

[9.9243212e-01 6.1247447e-03 9.3063264e-04]
['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']


## Distribución de probabilidades para:
#### "Estaba la novia"

In [95]:
probs = predict_phrase(model, "Estaba la novia")
indices_to_chars[sample_probs(probs, sample_type='greedy')]

[('.', 0.6997879), (' ', 0.20880213), (',', 0.037900005), ('\n', 0.029043293), ('!', 0.011357076)]


'.'

### Mismo ejemplo pero cambiar temperatura de ruido
Notar que muestramos

In [98]:
temp = 1
index, dist = sample(probs, temp, verbose=False, return_dist=True)
sorted_idxs = np.argsort(dist)[::-1]
print(dist[sorted_idxs][:3])
samples = []
for i in range(20):
    index, dist = sample_probs(probs, sample_type='simple', temp=temp)
    samples.append(indices_to_chars[index])
print(samples)

[0.69978786 0.2088021  0.0379    ]
['.', '.', '.', ' ', '.', '.', '.', '.', '.', '.', '.', '.', ' ', '.', '.', '.', ' ', ' ', ',', ' ']


## Generamos texto
### Greedy Search

In [51]:
def greedy_search(model, phrase, N=200, window=100):
    print(phrase, end='')
    for i in range(N):
        probs = predict_phrase(model, phrase, verbose=False)
        next_char = indices_to_chars[sample_probs(probs, sample_type='greedy')]
        phrase=phrase + next_char
        print(next_char, end='')

In [58]:
text = open('bodas_de_sangre.txt').read()

In [59]:
print(text[:100])

BODAS DE SANGRE(1933)

Poema trágico en tres actos y siete cuadros

Personajes
 
Madre
Criada
Leonar


In [60]:
greedy_search(model, text[:100])

BODAS DE SANGRE(1933)

Poema trágico en tres actos y siete cuadros

Personajes
 
Madre
Criada
Leonardo de caballo se pone a la boda.
MUJER: (Llorando)
Duérmete, clavel,
que el caballo se pane a la boda.
MUJER: (Llorando)
Duérmete, clavel,
que el caballo se pane a la boda.
MUJER: (Llorando)
Duérmete,

In [61]:
greedy_search(model, 'NOVIA:')

NOVIA: (Seria) Ahora de casaria. (Se siente de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de azahar.)

NOVIA: (Alegre) ¡Qué te puede a la carrel?

MADRE: Sí. (Se siente

In [62]:
greedy_search(model, 'LEONARDO:', 300)

LEONARDO: (Levantándose) ¡Qué casa?

MADRE: (Al novio) No te puede a me puede a la carre. Pero no se puede a me puede a la carre. Pero no se puede a me puede a la carre. Pero no se puede a me puede a la carre. Pero no se puede a me puede a la carre. Pero no se puede a me puede a la carre. Pero no se puede a 

In [63]:
greedy_search(model, 'JULIAN:', 300)

JULIAN:
Mujer de la novia.
MUJER: (Llorando)
Duérmete, clavel,
que el caballo se pala de la boda.
MUJER: (Llorando)
Duérmete, clavel,
que el caballo se pane a la boda.
MUJER: (Llorando)
Duérmete, clavel,
que el caballo se pane a la boda.
MUJER: (Llorando)
Duérmete, clavel,
que el caballo se pane a la boda.

In [64]:
# Con texto de la obra
initial_text = 'MOZO 1: (Entrando) ¡Tienes que beber con nosotros!\n\nNOVIO: Estoy esperando a la novia.\n\nMOZO 2: ¡Ya '
print(initial_text)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya 


In [27]:
greedy_search(model, initial_text, N=300)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya vinte a la cara con el caballo de azahar.)

NOVIA: (Alegre) ¡Qué te puede a la carrel?

MADRE: Sí. (Se siente en la cabeza) ¿Qué te puede a la carre?

MADRE: Sí. (Se siente en la cabeza) ¿Qué te puede a la carre?

MADRE: Sí. (Se siente en la cabeza) ¿Qué te puede a la carre?

MADRE: Sí. (Se siente e

In [28]:
greedy_search(model, 'MADRE:', 300)

MADRE: (Seria) Ahora está con la cabeza de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo 

## Random Sample

In [71]:
def simple_sample(model, phrase, temp=1, N=200):
    print(phrase, end='')
    for i in range(N):
        probs = predict_phrase(model, phrase, verbose=False)
        index, dist = sample_probs(probs, sample_type='simple', temp=temp)
        next_char = indices_to_chars[index]
        phrase=phrase + next_char
        print(next_char, end='')

In [75]:
simple_sample(model, 'NOVIA:', temp=1, N=300)

NOVIA: Dempierte de tierma.

PADRE: Vamos niño pero una des las rabaendose.

MADRE: ¿Pastá samarre ton casa?

NOVIA: Trencos de aharron telor fanga por la sosa. Pardo simo mu mija?

MADRE: Los poces nos de bien. Hablo, tragos me compro que solupo lo mare quié sé momido el hucho, lo una de caso de abrador 

In [76]:
# Cada pasada escribe algo diferente
simple_sample(model, 'NOVIA:', temp=1, N=300)

NOVIA:jO pero noche lo vablira, (al faliciandete la novia. se coronflemos a la boda en estermente, y lo culperta, que tu padres que un dírmisas di hijo noda estarmo las mantos, poro puede romprescido de ducho está delor de la grien... (Pausa)

CRIADA: ¿y tras más?

MADRE: ruo que me suegre. Vamos.

MADRE:

In [77]:
# Cambiando temperatura
simple_sample(model, 'NOVIA:', 0.1, 300)

NOVIA: (Seria) Ahora está con la cabeza de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo se mu hijo es una con la cabeza. Es estar tras de casa.

NOVIA: (Seria) ¿Qué te puede a la carrel?

MADRE: Sí. (Se siente en l

In [78]:
simple_sample(model, 'NOVIA:', 0.1, 300)

NOVIA: (Seria) ¿Qué te puede a la carrel?

MADRE: Sí. (Se siente de la caballo de azahar.)

NOVIA: (Al novio) No te puede este mujer.

MADRE: Sí. (Se siente de azahar.)

NOVIA: (Al niña) ¡Qué está que me puede a la cara?

NOVIA: (Alegre) ¡Y está con las mantes de azahar?

NOVIA: (Alegre) ¿Y tú?

NOVIO: (E

In [33]:
simple_sample(model, 'NOVIA:', 2)

NOVIA:fdéécano;yoqui miñan.
¡racá fEltade
la llegazóMañagiadde.)
PADRE:nfrefoco.

lajay
do me oprebhelcis;suñoudedojie.
Méagente llesól que Us compojelovia.Que monsel,
(uputa
¿visconte?
CUiER: Esfdí quesegá

In [34]:
simple_sample(model, initial_text, 0.5, 300)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya te puede con la cuerte de buena de viste a la orecha de despierte.)

MADRE: Pero la muchacha con la guente.

NOVIA: ¿Cuándo está a tu mujer?

CRIADA: No. La novia se mus hombre y a verta al mine.

MADRE: Sí, pero vendo el alunto de tierma y puede miento y la mejer.

MADRE: Ya no está espara.

NOVIA:

# Stochastic Beam search

https://www.youtube.com/watch?v=zPv7nWB4_ts

In [79]:
def init_models(beam_width):
    models = []
    for i in range(beam_width):
        model = get_deep_rnn((100, 69), dense_units=69, stateful=False, verbose=False)
        model.load_weights('bodes_sangre_LSTM_deep_dropout_04.hdf5')
        models.append(model)
    return models

In [80]:
def predict_beams(phrases, log_probs, beam_width, temp=1):
    samples = []
    log_probs_out = []
    for j, phrase in enumerate(phrases):
        for i in range(beam_width):
            probs = predict_phrase(models[i], phrase, verbose=False)
            index, dist = sample_probs(probs, sample_type='simple', temp=temp)
            samples.append(phrase + indices_to_chars[index])
            log_probs_out.append(log_probs[j] + np.log(dist[index]))
    return np.array(samples), np.array(log_probs_out)

In [81]:
beam_width = 5
models = init_models(beam_width)

In [86]:
initial_phrase = 'NOVIA:'
phrases = [initial_phrase]
log_probs = np.zeros(len(phrases)*beam_width)
for i in range(100):
    print(phrases, end=' ')
    phrases, log_probs = predict_beams(phrases, log_probs, beam_width, temp=2)
    print('Genero '+str(len(phrases))+' posibles y me quedo con los {} mas probables'.format(beam_width))
    #print(phrases)
    #print(log_probs)
    sorted_indexes = np.argsort(log_probs)[::-1][:beam_width]
    log_probs = log_probs[sorted_indexes]
    phrases = phrases[sorted_indexes]
    print('---------------------------------------------')
print(phrases[0])

['NOVIA:'] Genero 5 posibles y me quedo con los 5 mas probables
---------------------------------------------
['NOVIA:\n' 'NOVIA:¿' 'NOVIA:q' 'NOVIA:d' 'NOVIA:B'] Genero 25 posibles y me quedo con los 5 mas probables
---------------------------------------------
['NOVIA:\n¡' 'NOVIA:\n¡' 'NOVIA:\nD' 'NOVIA:\nN' 'NOVIA:qu'] Genero 25 posibles y me quedo con los 5 mas probables
---------------------------------------------
['NOVIA:\nDe' 'NOVIA:\nDu' 'NOVIA:que' 'NOVIA:que' 'NOVIA:\nNo'] Genero 25 posibles y me quedo con los 5 mas probables
---------------------------------------------
['NOVIA:\nDes' 'NOVIA:\nDué' 'NOVIA:que ' 'NOVIA:que ' 'NOVIA:que '] Genero 25 posibles y me quedo con los 5 mas probables
---------------------------------------------
['NOVIA:\nDesp' 'NOVIA:\nDesp' 'NOVIA:\nDuér' 'NOVIA:\nDuér'
 'NOVIA:\nDuém'] Genero 25 posibles y me quedo con los 5 mas probables
---------------------------------------------
['NOVIA:\nDespi' 'NOVIA:\nDespi' 'NOVIA:\nDespi' 'NOVIA:\nDespi'

In [41]:
print(phrases[0])

NOVIA: Vamos a la madre. Pero como una mujer.

MADRE: (Al novio) No tengo es un hombre. Pero es un caballo


## Algunas frases con Beam Search
Temp = 1

Temp = 2

NOVIA: Vamos a la madre. Pero como una mujer.

MADRE: (Al novio) No tengo es un hombre. Pero es un caballo

# Verificación Beam search

In [1101]:
selected_pred = 0 # Cual de las predicciones verifico. Deberia quedarme con la de mayor prob
predicted_chars = phrases[selected_pred][len(initial_phrase):]
print(predicted_chars)
log_sum = 0
predicted_phrase_parc = initial_phrase
for c in predicted_chars:
    pred_prob = predict_phrase(model, predicted_phrase_parc, verbose=False)[chars_to_indices[c]]
    print(pred_prob)
    predicted_phrase_parc = predicted_phrase_parc + c
    log_sum = log_sum + np.log(pred_prob)
log_sum

 (Ser
0.525927
0.296634
0.278172
0.86822
0.606687


-3.7784151881933212